In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv("./dataset.csv")

In [3]:
with open("./weaponUsedTransformer.pkl", "rb") as f:
    weaponUsedTransformer = pickle.loads(f.read())
with open("./weaponCategoryTransformer.pkl", "rb") as f:
    weaponCategoryTransformer = pickle.loads(f.read())

In [4]:
df2 = df.drop(['currentTick','playerId','playerName','X','Y','Z', 'yaw', 'pitch', 'targetId','targetName','targetX','targetY','targetZ'], axis=1)

In [5]:
def fights(df: pd.DataFrame):
    indexes: list = [-1]
    indexes.extend(df[df.isnull().all(axis=1)].index.to_list())
    for i in range(1, len(indexes)):
        prevRowIndex: int = int(indexes[i - 1] + 1)
        curRowIndex: int = int(indexes[i])
        curDf: pd.DataFrame = df.iloc[prevRowIndex: curRowIndex, :].copy()
        curDf.reset_index(inplace = True)
        target_column = 'Label'
        target = df2[target_column].iloc[prevRowIndex]
        curDf.drop(target_column, axis=1, inplace= True)
        # dfs.append(curDf)
        yield curDf, target

In [6]:
lsdf = list(fights(df2))

In [7]:
len(lsdf)

109746

In [8]:
lsdf[0][0]

,index,deltaX,deltaY,deltaZ,deltaYaw,deltaPitch,deltaAimArc,isFlashed,isCrouching,isInAir,utilityDmgDone,supportUtilityUsed,KDR,isFiring,targetDeltaX,targetDeltaY,targetDeltaZ,isHurt,dmgDone,distToTarget,targetHitArea,weaponUsed,weaponCategory,isScoping,isTargetBlind,shotTargetThroughSmoke,targetReturnedDmg
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,False,0.0,2.0,1.0,False,0.000000,0.000000,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-0.078308,-0.044434,-0.026367,-1.323853,-0.318604,28.367904,0.0,0.0,False,0.0,2.0,1.0,False,-0.606720,-0.130192,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.015717,-0.037720,0.000000,-1.263428,-0.351562,27.313905,0.0,0.0,False,0.0,2.0,1.0,False,-0.506760,-0.132717,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.077820,-0.021118,0.031250,-0.758057,-0.247192,16.602574,0.0,0.0,False,0.0,2.0,1.0,False,-0.407074,-0.134075,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.125092,-0.021851,0.041992,-0.543823,-0.247192,12.437608,0.0,0.0,False,0.0,2.0,1.0,False,-0.307831,-0.133984,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,0.171814,-0.025391,0.056641,-0.373535,-0.241699,9.264133,0.0,0.0,False,0.0,2.0,1.0,False,-0.209442,-0.131790,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,0.218414,-0.029907,0.073242,-0.208740,-0.318604,7.938828,0.0,0.0,False,0.0,2.0,1.0,False,-0.112854,-0.126263,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,0.193268,-0.026489,0.064453,-0.038452,-0.263672,5.558402,0.0,0.0,False,0.0,2.0,1.0,False,-0.020828,-0.115353,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,0.168091,-0.023071,0.055664,-0.049438,-0.318604,6.725509,0.0,0.0,False,0.0,2.0,1.0,True,0.058792,-0.098392,0.0,True,28.0,1195.281448,2.0,weapon_galilar,weapon_category_ar,False,0.0,False,113.0
9,9,0.142944,-0.019531,0.046875,-0.054932,-0.225220,4.835008,0.0,0.0,False,0.0,2.0,1.0,False,0.120865,-0.084618,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
lsdf[0][1]

False

In [10]:
X = []
Y = []

i = 0
for features, target in lsdf:
    features.fillna(-1, inplace=True)

    features['isInAir'] = features['isInAir'].astype(int).values
    features['isFiring'] = features['isFiring'].astype(int).values
    features['isHurt'] = features['isHurt'].astype(int).values
    features['isScoping'] = features['isScoping'].astype(int).values
    features['shotTargetThroughSmoke'] = features['shotTargetThroughSmoke'].astype(int).values
        
    features["deltaX"] = features["deltaX"].astype(float).values / 64
    features["deltaY"] = features["deltaY"].astype(float).values / 64
    features["deltaZ"] = features["deltaZ"].astype(float).values / 64
    features["deltaYaw"] = features["deltaYaw"].astype(float).values / 64
    features["deltaPitch"] = features["deltaPitch"].astype(float).values / 64
    features["deltaAimArc"] = features["deltaAimArc"].astype(float).values / 64
    features["utilityDmgDone"] = features["utilityDmgDone"].astype(float).values / 64
    features["targetDeltaX"] = features["targetDeltaX"].astype(float).values / 64
    features["targetDeltaY"] = features["targetDeltaY"].astype(float).values / 64
    features["targetDeltaZ"] = features["targetDeltaZ"].astype(float).values / 64
    features["dmgDone"] = features["dmgDone"].astype(float).values / 64
    features["distToTarget"] = features["distToTarget"].astype(float).values / 128
    features["targetReturnedDmg"] = features["targetReturnedDmg"].astype(float).values / 64
    
    features['weaponUsed'] = features['weaponUsed'].replace(-1, "none")
    features['weaponCategory'] = features['weaponCategory'].replace(-1, "none")
        
    features['weaponUsed'] = weaponUsedTransformer.transform(features['weaponUsed'])
    features['weaponCategory'] = weaponCategoryTransformer.transform(features['weaponCategory'])

    X.append(features)
    Y.append(target)

In [11]:
X[0].to_numpy()[:, :, np.newaxis].shape

(13, 27, 1)

In [12]:
max_rows = max(df.shape[0] for df in X)
print("Max Rows:", max_rows)

Max Rows: 16


In [13]:
num_samples = len(X)
num_columns = X[0].shape[1]
data_array = np.zeros((num_samples, max_rows, num_columns))

# Iterate over the DataFrames and fill the array
for i, df in enumerate(X):
    data_array[i, :df.shape[0], :] = df.values

In [14]:
data_array.shape

(109746, 16, 27)

In [15]:
np.save('X.npy', data_array)

In [16]:
Y_array = np.array(Y)

In [17]:
np.save('Y.npy', Y_array)

In [18]:
df2['Label'].value_counts()

Label
False    1088457
True      270314
Name: count, dtype: int64